# Apprenez une heuristique pour le Go

Dans ce **TP noté**, vous devrez déployer des methodes d'apprentissage automatique permettant d'évaluer la qualité de plateaux de GO.

Pour cela, vous disposerez de 21854 exemples de plateau de Go, tous générés par `gnugo` après quelques coups contre lui même avec un niveau de difficulté de 0. Par chaque plateau, nous avons lancé 100 matchs de gnugo contre lui même, toujours avec un niveau 0, et compté le nombre de victoires de noir et de blanc depuis ce plateau.

A noter, chaque "rollout" (un rollout et un déroulement possible du match depuis le plateau de référence) correspond à des mouvements choisis aléatoirement parmis les 10 meilleurs mouvements possibles, en biasant le choix aléatoire par la qualité prédite du mouvement par gnugo (les meilleurs mouvements ont une plus forte probabilité d'être tirés).

Les données dont vous disposez sont brutes. Ce sera à vous de proposer un format adéquat pour utiliser ces données en entrée de votre réseau neuronal.


## Comment sera évalué votre modèle ?

Nous vous fournirons 6h avant la date de rendu un nouveau fichier contenant 1000 nouveaux exemples, qui ne contiendront pas les champs `black_wins`, `white_wins`, `black_points` et `white_points`. Vous devrez laisser, dans votre dépot de projet (votre dépot GIT sous un sous-répertoire ML) un fichier texte nommé `my_predictions.txt` ayant une prédiction par ligne (un simple flottant) qui donnera, dans le même ordre de la liste des exemples les scores que vous prédisez pour chacune des entrées du fichier que nous vous aurons donné. ** Les scores seront donnés sous forme d'un flottant, entre 0 et 1, donnant la probabilité de victoire de noir sur le plateau considéré **. Il faudra laisser, dans votre feuille notebook (voir tout en dessous) la cellule Python qui aura créé ce fichier, pour que l'on puisse éventuellement refaire vos prédictions.

Bien entendu, vous nous rendrez également votre feuille jupyter **sous deux formats**, à la fois le fichier `.ipynb` et le fichier `.html` nous permettant de lire ce que vous avez fait, sans forcément relancer la feuille. Nous prendrons en compte les résultats obtenus sur les prédictions mais aussi le contenu de vos notebooks jupyter.

### Comment sera noté ce TP ?

**Il s'agit d'un TP noté (et non pas d'un projet), donc il ne faudra pas y passer trop de temps**. Nous attendons des prédictions correctes mais surtout des choix justifiés dans la feuille. Votre feuille notebook sera le plus important pour la notation (n'hésitez pas à mettre des cellules de texte pour expliquer pourquoi vous avez été amenés à faire certains choix). Ainsi, il serait bien d'avoir, par exemple, les données (graphiques ou autre) qui permettent de comprendre comment vous avez évité l'overfitting.

Le fichier de vos prédiction sera évalué en se basant sur la qualité de vos prédictions. Nous mesurerons par exemple (juste pour vous donner une idée) le nombre d'exemples dont votre prédiction donnera la bonne valeur à 5%, 10%, 20%, 35%, 50% pour estimer sa qualité.


## Mise en route !

Voyons  comment lire les données


In [2]:
# Import du fichier d'exemples
import numpy as np

def get_processed_data_go():
	import json
	with open('database-8x8.json', 'r') as f:
		return json.load(f)

data = get_processed_data_go()
print("We have", len(data),"examples")

def prepare_data(data):
	sizetraining = int(len(data) * 0.33)
	black = []
	white = []
	for games in data:
		if games["black_wins"]/100 >= 0.5:
			black.append(games)
		else:
			white.append(games)
	nbblack = len(black)
	nbwhite = len(white)

	print("white:", nbwhite, "black:", nbblack)

	while nbblack > nbwhite:
		shuffledwhite = white.copy()
		white = np.concatenate((white, shuffledwhite[:nbblack - nbwhite]))
		nbwhite = len(white)
		np.random.shuffle(white)

	while nbwhite > nbblack:
		shuffledblack = black.copy()
		black = np.concatenate((black, shuffledblack[:nbwhite - nbblack]))
		nbblack = len(black)
		np.random.shuffle(black)

	print("white:", nbwhite, "black:", nbblack)

	train_white = white[:sizetraining]
	val_white = white[sizetraining:]
	train_black = black[:sizetraining]
	val_black = black[sizetraining:]
	train_data = np.concatenate((train_black, train_white))
	np.random.shuffle(train_data)
	val_data = np.concatenate((val_black, val_white))
	np.random.shuffle(val_data)
	return train_data, val_data

size = 30000
new_datas = prepare_data(data)

We have 664848 examples
white: 355600 black: 309248
white: 355600 black: 355600


## Compréhension des données de chaque entrée

Voici une description de chaque exemple

In [3]:
def summary_of_example(data, sample_nb):
    ''' Gives you some insights about a sample number'''
    sample = data[sample_nb]
    print("Sample", sample_nb)
    print()
    print("Données brutes en format JSON:", sample)
    print()
    print("The sample was obtained after", sample["depth"], "moves")
    print("The successive moves were", sample["list_of_moves"])
    print("After these moves and all the captures, there was black stones at the following position", sample["black_stones"])
    print("After these moves and all the captures, there was white stones at the following position", sample["white_stones"])
    print("Number of rollouts (gnugo games played against itself from this position):", sample["rollouts"])
    print("Over these", sample["rollouts"], "games, black won", sample["black_wins"], "times with", sample["black_points"], "total points over all this winning games")
    print("Over these", sample["rollouts"], "games, white won", sample["white_wins"], "times with", sample["white_points"], "total points over all this winning games")

# summary_of_example(data,10)

## Données en entrée et en sortie de votre modèle final

Même si en interne, votre modèle va manipuler des tenseurs en numpy, vous devrez construire une boite noire qui prendra en entrée les données dans le style du JSON ci-dessous. Typiquement, vous aurez le même genre de fichier avec seulement les champs `black_stones`, `white_stones`, `depth` et `list_of_moves` de renseignées. Vous devrez utiliser ces champs, dont notemment les coordonnées des pierres noires et blanches et donner le pourcentage de chance pour noir de gagner depuis cette position.

Ainsi, pour l'exemple `i` :
- Vous pourrez prendez en entree `data[i]["black_stones"]` et `data[i]["white_stones"]` (vous pouvez, si vous le souhaitez, prendre en compte également `list_of_moves` ou tout autre donnée que vous calculerez vous-même (mais qui ne se basera évidemment pas sur les données que vous n'aurez pas lors de l'évaluation).
- Vous devrez prédire simplement `data[i]["black_wins"]/data[i]["rollouts"]` en float (donc une valeur entre 0 et 1).

Encore une fois, **attention** : en interne, il faudra absolument construire vos données formattées en matrices numpy pour faire votre entrainement. On vous demande juste ici d'écrire comment vous faites ces transformations, pour comprendre ce que vous avez décidé de mettre en entrée du réseau.

Voici par exemple le modèle de la fonction qui pourra être appelée, au final :


In [4]:
def position_predict(black_stones, white_stones):

    # ... Votre tambouille interne pour placer les pierres comme il faut dans votre structure de données
    # et appeler votre modèle Keras (typiquement avec model.predict())
    prediction = None # model.predict(...) # A REMPLIR CORRECTEMENT

    return prediction

# Par exemple, nous pourrons appeler votre prédiction ainsi

# print("Prediction this sample:")
# data_index = 1
# summary_of_example(data, data_index)
# print()
# prediction = position_predict(data[data_index]["black_stones"], data[data_index]["white_stones"])
# print("You predicted", prediction, "and the actual target was", data[data_index]["black_wins"]/data[data_index]["rollouts"])

# Ainsi, pour le rendu, en admettant que newdata soit la structure de données issue du json contenant les nouvelles données que
# l'on vous donnera 24h avant la fin, vous pourrez construire le fichier resultat ainsi

def create_result_file(newdata):
    ''' Exemple de méthode permettant de générer le fichier de resultats demandés. '''
    resultat  = [position_predict(d["black_stones"], d["white_stones"]) for d in newdata]
    with open("my_predictions.txt", "w") as f:
         for p in resultat:
            f.write(str(p)+"\n")



In [5]:
# %matplotlib inline
# import matplotlib.pyplot as plt

# plt.title("Relationship between the depth of the board and the chance for black to win")
# plt.plot([sample["black_wins"] for sample in data],[sample["depth"] for sample in data], '.')
# plt.xlabel("black wins (percentage)")
# plt.ylabel("depth of the game")


# # Cumulative Distribution function of the chance of black to win
# cdf_wins = sorted([sample["black_wins"] for sample in data])
# plt.figure()
# plt.plot([x/len(cdf_wins) for x in range(len(cdf_wins))], cdf_wins)
# plt.grid()
# plt.title("Cumulative Distribution function of the chance of black to win")
# plt.xlabel("% of the samples with a chance of black to win below the y value")
# plt.ylabel("Chance of black to win")
# print("The CDF curve shows that black has more chances to win, globally")

# First steps: transform all the data into numpy arrays to feed your neural network

Advices:
- do not use only a 9x9 matrix as input. Use at least two planes to encode the board. One plane for black and one plane for white (typically with a 1 if there is a black stone for the first plane and with a 1 if there is a white stone for the second plane). The dimension of an input should be at least `[2,9,9]`. In Torch, the Conv2d method needs inputs as `[NBatch, Channels, H, W]`.
- consider to enrich your dataset with all symmetries and rotations. You should be able to multiply the number of samples to consider: any rotation of the board should have the same score, right?. You can use `np.rot90` to rotate your boards be beware of the dimensions (the channel is not the last dimension), so you may want to use `np.moveaxis()` to force the channels to be the last dimension, then call it again to make it the second one.
- what should happen on the score if you switch the colors? To know which player has to play next, you can check, for a sample, the parity of the length of the list `data[i]["list_of_moves"]` (an odd length list would mean that white is the next player. An even length list means that black has to play).
- work on enlarging and preparing your data only once. Once all you input data is setup as a big Numpy matrix, you may want to save it for speeding up everything. You can use, for instance `numpy.rot90()` and `numpy.flipud()` to generate all the symmetries



In [6]:
import numpy as np

# Convert to tensor that the ML will take as input
def coord_to_board(black_coord, white_coord):
    board = np.zeros((2,8,8), dtype=np.double)
    for black in black_coord:
        board[0][(black[0],black[1])] = 1
    for white in white_coord:
        board[1][(white[0], white[1])] = 1
    return np.array(board)

print(coord_to_board(data[0]["black_coord"], data[0]["white_coord"]))




[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 1. 1. 0.]
  [0. 0. 1. 0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0. 1. 0. 1.]
  [0. 0. 0. 0. 0. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]]


# Second steps: build your neural network and train it

Don't forget to check overfitting, ...

*advices* :
- you may need to use some of the `torch.nn` layers: `Linear`, `Conv2d`, `ReLU`, `LeakyReLU`, `BatchNorm2d`, `Flatten`, `Dropout`... But you can of course first build a very simple one (and just pick some of them)...
- if you use convolution layers, be sure **not to downsize your board**. Applying a filter should keep the original size of the board (9x9), otherwise you would somehow forget the stones on the borders
- you will use like 33% of your input sample for validation. However, the final goal is to score new data that will be given in addition to the actual data. So, you should use the 33% splitting rule to set up your network architecture and, once you fixed it, you should train your final model on the whole set of data, crossing your fingers that it will generalize well.
- Warning: if you run a few epoch, and run it again for some more epochs, it will not reset the weights and the biases of your neural network. It's good news because you can add more and more epochs to your model, but be careful about the training/test sets (do split your sets before you initialize your model). Or you will be breaking your validation/training partition!


In [19]:
import torch
import torch.nn.functional as F
from torch.nn import Sequential, Conv2d, Tanh, ReLU, Dropout, Linear, MaxPool2d
import torch.optim as optim
import torch.nn as nn

if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

def train(model, device, train_loader,  optimizer, epoch):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device).float(), target.to(device).float()
        target = target.view(-1, 1)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output, target, reduction="mean") # Read the documentation.
        total_loss += loss.item() * len(data) # We average the loss
        loss.backward()
        optimizer.step()
    total_loss /= len(train_loader.dataset)
    return {"loss": total_loss}

def test(model, device, test_loader):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device).float(), target.to(device).float()
            target = target.view(-1, 1)
            output = model(data)
            loss = F.binary_cross_entropy(output, target, reduction="sum")
            total_loss += loss.item()
            predictions = torch.round(output * 100) / 100
            correct += predictions.eq(target.view_as(predictions)).sum().item()
    total_loss /= len(test_loader.dataset)
    return {"loss": total_loss, "correct %": int(10000*correct/len(test_loader.dataset))/100}



class GoFindWin(nn.Module):
  def __init__(self):
    super(GoFindWin, self).__init__()
    self.flatten = nn.Flatten(start_dim=1) # Do not flatten the batch dimension

    self.conv1 = Conv2d(2, 8, 3, padding="same")
    self.conv2 = Conv2d(8, 20, 3, padding="same")
    self.sub1 = MaxPool2d(2)
    self.flatten = nn.Flatten(start_dim=1)

    self.dense1 = nn.Linear(20*4*4, 32)
    self.dense2 = nn.Linear(32, 1)
    

  def forward(self, x):
    x = self.conv1(x)
    x = F.tanh(x)
    x = self.conv2(x)
    x = F.tanh(x)
    x = self.sub1(x)
    x = self.flatten(x)
    x = self.dense1(x)
    x = F.tanh(x)
    x = self.dense2(x)
    out = F.sigmoid(x) # In torch, we can either use log_softmax + nll_loss or the CrossEntropy (without the last layer of softmax)
    return out


def format_data(datas):
    datas_length = len(datas)
    X = []
    Y = np.zeros(datas_length, dtype=np.double)
    for i in range(datas_length):
        X.append(coord_to_board(datas[i]["black_coord"], datas[i]["white_coord"]))
        Y[i] = datas[i]["black_wins"]/100
    return np.array(X), Y

model = torch.load("model.pth")
# model = GoFindWin().to(device).float()

# print(model)

batch_size = 32

train_data, test_data = prepare_data(data[:1000])

train_x, train_y = format_data(train_data)
test_x, test_y = format_data(train_data)
# print(train_data[0])
# print(train_x[0], train_y[0])
# print(test_x[0], test_y[0])
trainloader = torch.utils.data.DataLoader(list(zip(train_x, train_y)), batch_size=batch_size, shuffle=True, num_workers=1)
testloader = torch.utils.data.DataLoader(list(zip(test_x, test_y)), batch_size=batch_size, shuffle=False, num_workers=1)
print(len(testloader))

optimizer = optim.Adam(model.parameters(), lr=0.01)
for epoch in range(1, 20):
  train_stats = train(model, device, trainloader, optimizer, epoch)
  test_stats = test(model, device, testloader)
  print(train_stats, test_stats)

torch.save(model, "model.pth")


white: 528 black: 472
white: 528 black: 528
21
{'loss': 0.3339519016670458} {'loss': 0.30940243619861024, 'correct %': 24.24}
{'loss': 0.3071307636571653} {'loss': 0.3032846855394768, 'correct %': 21.21}
{'loss': 0.2983791784806685} {'loss': 0.29591834834127717, 'correct %': 26.21}
{'loss': 0.2921107010407881} {'loss': 0.2862176360506, 'correct %': 33.18}
{'loss': 0.2864848427700274} {'loss': 0.2829976529786081, 'correct %': 39.09}
{'loss': 0.28390628334247703} {'loss': 0.28359392700773295, 'correct %': 37.87}
{'loss': 0.28249978043816304} {'loss': 0.28102580345038214, 'correct %': 44.84}
{'loss': 0.28122228943940364} {'loss': 0.28069913676290803, 'correct %': 45.15}
{'loss': 0.2807339104739102} {'loss': 0.2804088260188247, 'correct %': 48.93}
{'loss': 0.2804478605588277} {'loss': 0.2803544008370602, 'correct %': 53.63}
{'loss': 0.2802241092378443} {'loss': 0.2799546256209865, 'correct %': 54.54}
{'loss': 0.28012362610210073} {'loss': 0.2798281546795007, 'correct %': 55.3}
{'loss': 0.2

# Last step

Prepare your model to predict the set of new data to predict, you will have only 6 hours to push your predictions.

(may be you would like to express, when guessing the percentage of wins for blacks, that it should reflect the fact that this score should be the same for all the symmetries you considered)...